In [10]:
import numpy as np
import nltk

def get_letter_freq(words,letter):
    return [elem[1] for elem in words if elem[0] == letter][0]


alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
punctuation = ['?',',','!','.',':',';']


#--------------------------------------------------------------------------------------------

### CHARS ###

input_file = sc.textFile("training_set/Abraham Lincoln/Abraham Lincoln___Lincoln Letters.txt")

letters = (
            input_file
            .flatMap(lambda line: list(line.lower().replace(" ","")))
            .map(lambda x: (x, 1))
            .reduceByKey(lambda freq1, freq2: freq1 + freq2)
            .sortByKey(True)
         )
letters = letters.collect()

tot_chars = sum(elem[1] for elem in letters)
tot_punctuation = sum(elem[1] for elem in letters if elem[0] in punctuation)

alphabet_frequency = np.array([get_letter_freq(letters,letter) for letter in alphabet])
alphabet_frequency=alphabet_frequency/tot_chars

punctuation_frequency = np.array([get_letter_freq(letters,punct) for punct in punctuation])
punctuation_frequency =punctuation_frequency/tot_punctuation

### WORDS ###

words = (
            input_file
            .flatMap(lambda x: nltk.word_tokenize(x))
            .map(lambda x: (nltk.pos_tag([x],tagset='universal')[0][1],1))
            .reduceByKey(lambda freq1, freq2: freq1 + freq2)
            .sortByKey(True)
        )

pos_frequency = words.collect()

pos_ratios = [0] * 8

for elem in pos_frequency:
        if 'DET' == elem[0]:
            pos_ratios[0] += elem[1]
        elif 'ADP' == elem[0]:
            pos_ratios[1] += elem[1]
        elif 'PRON' == elem[0]:
            pos_ratios[2] += elem[1]
        elif 'VERB' == elem[0]:
            pos_ratios[3] += elem[1]
        elif 'ADV' == elem[0]:
            pos_ratios[4] += elem[1]
        elif 'CONJ' == elem[0]:
            pos_ratios[5] += elem[1]
        elif 'NOUN' == elem[0]:
            pos_ratios[6] += elem[1]
        elif 'ADJ' == elem[0]:
            pos_ratios[7] += elem[1]
            
            
tot_words = sum(elem[1] for elem in pos_frequency)

pos_ratios = np.array(pos_ratios)
pos_ratios = pos_ratios/tot_words

lexical_variety = sum(elem[1] for elem in pos_frequency if (elem[0]=='NOUN' or 
                                                            elem[0]=='ADJ'  or 
                                                            elem[0]=='VERB' or 
                                                            elem[0]=='ADV' ) )

tot_sentences = len(nltk.sent_tokenize(open("training_set/Abraham Lincoln/Abraham Lincoln___Lincoln Letters.txt","r").read()))

avg_sentence_len = tot_words / tot_sentences

avg_word_len = tot_chars / tot_words

feature_vector = []

feature_vector.extend(alphabet_frequency)
feature_vector.extend(punctuation_frequency)
feature_vector.extend(pos_ratios)
feature_vector.extend([lexical_variety])
feature_vector.extend([avg_sentence_len])
feature_vector.extend([avg_word_len])
feature_vector.extend([tot_words])

feature_vector = np.array(feature_vector).reshape(-1,1)
print(feature_vector.shape)
print(feature_vector)

(44, 1)
[[6.98433420e-02]
 [1.45778938e-02]
 [2.00174064e-02]
 [2.98085292e-02]
 [1.01392515e-01]
 [2.24107920e-02]
 [1.54482158e-02]
 [4.76501305e-02]
 [7.09312446e-02]
 [1.52306353e-03]
 [6.52741514e-03]
 [4.76501305e-02]
 [2.04525674e-02]
 [6.54917319e-02]
 [9.33420366e-02]
 [1.47954743e-02]
 [1.30548303e-03]
 [5.67885117e-02]
 [4.80852916e-02]
 [9.11662315e-02]
 [3.80765883e-02]
 [1.15317668e-02]
 [2.21932115e-02]
 [1.95822454e-03]
 [3.74238468e-02]
 [2.17580505e-04]
 [6.71140940e-03]
 [5.43624161e-01]
 [6.71140940e-03]
 [3.42281879e-01]
 [6.04026846e-02]
 [4.02684564e-02]
 [7.25806452e-02]
 [1.15322581e-01]
 [1.28225806e-01]
 [1.25000000e-01]
 [6.85483871e-02]
 [3.30645161e-02]
 [2.50806452e-01]
 [3.87096774e-02]
 [5.99000000e+02]
 [2.63829787e+01]
 [3.70645161e+00]
 [1.24000000e+03]]
